In [1]:
import pymysql
import decouple
import requests
import json
import os
import re
import pandas as pd
import numpy as np
import time


In [266]:
config = decouple.AutoConfig(' ')
path = config('PATH_MY_MAIN_DATA')

In [265]:
# get tickers
config = decouple.AutoConfig(' ')
path = config('PATH_MY_MAIN_DATA')
active = pd.read_csv(path + '/symbols/listing_status.csv', parse_dates=True)
active = active[active['ipoDate'] <= '2005-12-01']
active_list = list(active['symbol'])
active_list

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dylanjorling/UCLA/mas_thesis/data/symbols/listing_status.csv'

In [64]:
#get sublists and combine
etfs = pd.read_csv(path + '/symbols/top_etfs.csv')
etf_list = list(etfs['Symbol'])
com_etfs = pd.read_csv(path + '/symbols/com_etf.csv')
com_etf_list = list(com_etfs['Ticker'])
bonds = pd.read_csv(path + '/symbols/bonds.csv')
bond_list = list(bonds['Symbol'])
ndx = pd.read_csv(path + '/symbols/ndx.csv')
ndx_list = list(ndx['Symbol'])
reit = pd.read_csv(path + '/symbols/reits.csv')
reit_list = list(reit['Symbol'])
vol = pd.read_csv(path + '/symbols/vol.csv')
vol_list = list(vol['Ticker'])
sp500 = pd.read_csv(path + '/symbols/sp500.csv')
sp500_list = list(sp500['Symbol'])

target_list = etf_list + com_etf_list + bond_list + ndx_list + reit_list + vol_list + sp500_list
target_list = list(np.unique(np.asarray(target_list)))



['A',
 'AAAU',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABMD',
 'ABNB',
 'ABT',
 'ACGL',
 'ACN',
 'ACWI',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AGG',
 'AGQ',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMJ',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ASML',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AZN',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAR',
 'BAX',
 'BBRE',
 'BBWI',
 'BBY',
 'BCD',
 'BCI',
 'BDX',
 'BEN',
 'BF.B',
 'BIIB',
 'BIL',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BMY',
 'BND',
 'BNDX',
 'BNO',
 'BOIL',
 'BR',
 'BRK.B',
 'BRO',
 'BSV',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDAY',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CMDY',
 'CME',
 

In [263]:
# get tickers and save to file
symbols = []
for t in target_list:
    if t in active_list:
        symbols.append(t)
# add a few extra symbols:
symbols = symbols + ['VIXM', 'UNG', 'USO', 'MCHI', 'EWJ']

drop_indices = []
drop_symbols = ['KIM', 'RHI', 'SPGI', 'WBD', 'WELL']
for s in drop_symbols:
    x = symbols.index(s)
    drop_indices.append(x)

keep_indices = []
for i in range(len(symbols)):
    if i in drop_indices:
        pass
    else:
        keep_indices.append(i)

symbols = [symbols[i] for i in keep_indices]


x = pd.DataFrame(data=symbols)

x.to_csv(path_or_buf=path+'/_data_symbols')


In [271]:
x = pd.read_csv(path+'/_data_symbols')
symbols = list(x.iloc[:,1])
len(symbols)

469

In [272]:
# scrape data
path = config('PATH_MY_MAIN_DATA')

day_count = 480
for symbol in symbols:
    api_key = config('ALPHAVANTAGE_API_KEY')
    size = 'full' # compact returns last 100, full returns 20+ years
    url_base = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='
    url = url_base + symbol + '&outputsize=' + size + '&apikey=' + api_key

    f = path + "/data_" + symbol + ".json"

    if os.path.isfile(f) == False:
        r = requests.get(url)
        data = r.json()
        data = json.dumps(data)
        with open(f, "w") as outfile:
            outfile.write(data)
        day_count -= 1
        if day_count % 5 == 0:
            time.sleep(60)
        if day_count == 0:
            break


In [298]:
df_list = []
for symbol in symbols:
    file = path + "/data_" + symbol + ".json"
    f = open(file)
    data = json.load(f)
    keys = list(data["Time Series (Daily)"].keys())
    
    df = pd.DataFrame(columns = ['date', 'open', 'high', 'low',
                                 'close', 'adjclose', 'volume', 'dividend',
                                 'splits', 'ticker'])
    for i, key in enumerate(keys):
        value_list = [key] + list(data["Time Series (Daily)"][key].values())
        value_list.append(symbol)
        df.loc[i] = value_list
    
    df = df.set_index('date')
    df_list.append(df)
    
    print(symbol + ' done')
   

A done
AAL done
AAP done
AAPL done
ABC done
ABMD done
ABT done
ACGL done
ACN done
ADBE done
ADI done
ADM done
ADP done
ADSK done
AEE done
AEP done
AES done
AFL done
AGG done
AIG done
AIZ done
AJG done
AKAM done
ALB done
ALGN done
ALK done
ALL done
AMAT done
AMD done
AME done
AMGN done
AMP done
AMT done
AMZN done
ANSS done
AON done
AOS done
APA done
APD done
APH done
ARE done
ASML done
ATO done
ATVI done
AVB done
AVY done
AXP done
AZN done
AZO done
BA done
BAC done
BALL done
BAX done
BBWI done
BBY done
BDX done
BEN done
BIIB done
BIO done
BK done
BKNG done
BLK done
BMY done
BRO done
BSX done
BWA done
BXP done
C done
CAG done
CAH done
CAT done
CB done
CBRE done
CCI done
CCL done
CDNS done
CE done
CF done
CHD done
CHRW done
CI done
CINF done
CL done
CLX done
CMA done
CMCSA done
CME done
CMI done
CMS done
CNC done
CNP done
COF done
COO done
COP done
COST done
CPB done
CPRT done
CPT done
CRL done
CRM done
CSCO done
CSGP done
CSX done
CTAS done
CTRA done
CTSH done
CVS done
CVX done
D done
DD

In [299]:
for df in df_list:
    for col in df.columns[:-1]:
        df[col] = pd.to_numeric(df[col])
    df['volume'] = df['volume'].astype('float')
    

In [300]:
count = len(df_list)
for df in df_list:    
    df['lag']  = df['adjclose'].shift(-1)
    df['return'] = (df['adjclose'] - df['lag']) / df['adjclose']
    df['norm_adjclose'] = df['adjclose']/ df['adjclose'][-1] * 100
    df['norm_adjopen'] = df['open'] / df['close'] * df['adjclose'] / df['adjclose'][-1] * 100
    df['norm_adjhigh'] = df['high'] / df['close'] * df['adjclose'] / df['adjclose'][-1] * 100
    df['norm_adjlow'] = df['low'] / df['close'] * df['adjclose'] / df['adjclose'][-1] * 100
    df['norm_adj_range_pct'] = (df['norm_adjhigh'] - df['norm_adjlow']) / df['norm_adjopen']
    df['volume_dollars_MM'] = (df['volume'] * ((df['low'] + df['high']) / 2) / 1000000).round(4)
    df['return_next'] = df['return'].shift(1)
    count -= 1
    print('{} dataframes remaining'.format(count))

468 dataframes remaining
467 dataframes remaining
466 dataframes remaining
465 dataframes remaining
464 dataframes remaining
463 dataframes remaining
462 dataframes remaining
461 dataframes remaining
460 dataframes remaining
459 dataframes remaining
458 dataframes remaining
457 dataframes remaining
456 dataframes remaining
455 dataframes remaining
454 dataframes remaining
453 dataframes remaining
452 dataframes remaining
451 dataframes remaining
450 dataframes remaining
449 dataframes remaining
448 dataframes remaining
447 dataframes remaining
446 dataframes remaining
445 dataframes remaining
444 dataframes remaining
443 dataframes remaining
442 dataframes remaining
441 dataframes remaining
440 dataframes remaining
439 dataframes remaining
438 dataframes remaining
437 dataframes remaining
436 dataframes remaining
435 dataframes remaining
434 dataframes remaining
433 dataframes remaining
432 dataframes remaining
431 dataframes remaining
430 dataframes remaining
429 dataframes remaining


133 dataframes remaining
132 dataframes remaining
131 dataframes remaining
130 dataframes remaining
129 dataframes remaining
128 dataframes remaining
127 dataframes remaining
126 dataframes remaining
125 dataframes remaining
124 dataframes remaining
123 dataframes remaining
122 dataframes remaining
121 dataframes remaining
120 dataframes remaining
119 dataframes remaining
118 dataframes remaining
117 dataframes remaining
116 dataframes remaining
115 dataframes remaining
114 dataframes remaining
113 dataframes remaining
112 dataframes remaining
111 dataframes remaining
110 dataframes remaining
109 dataframes remaining
108 dataframes remaining
107 dataframes remaining
106 dataframes remaining
105 dataframes remaining
104 dataframes remaining
103 dataframes remaining
102 dataframes remaining
101 dataframes remaining
100 dataframes remaining
99 dataframes remaining
98 dataframes remaining
97 dataframes remaining
96 dataframes remaining
95 dataframes remaining
94 dataframes remaining
93 dat

In [301]:
# check for missing indices
def  compare_index(df1, df2):
    if len(df1.index) >= len(df2.index):
        ind =  df2.index[-1]
        df = df1[df1.index >= ind]
       
    else:
        ind = df1.index[-1]
        df = df2[df2.index >= ind]
        
    if df2.index.equals(df.index):
        return True
    else:
        return False

unmatched = []
for df in df_list:
    if compare_index(df_list[3], df) == False:
        unmatched.append(df)

for df in unmatched:
    print(df['ticker'][0])

ABMD


In [302]:
#drop values w missing indices since only five...will be 468 ttl
drop_indices = []
drop_symbols = ['ABMD']
for s in drop_symbols:
    x = symbols.index(s)
    drop_indices.append(x)

keep_indices = []
for i in range(len(symbols)):
    if i in drop_indices:
        pass
    else:
        keep_indices.append(i)

df_list = [df_list[i] for i in keep_indices]


In [304]:
def concat_save_data(dfs, write_files=True):
    data_set_full = pd.DataFrame()
    data_set_model = pd.DataFrame()
    
    for df in dfs:
            df = df.add_suffix("_" + df['ticker'][0])
            data_set_full = data_set_full.join(df,
                                               how='outer').sort_index(ascending=False)
    
    df_model = []
    i = 0      
    for n in range(len(dfs)):
        df_model.append(data_set_full.iloc[:, (10+i):(18+i)])
        
        i += 18
    
    for df in df_model:
        if len(data_set_model) ==0:
            data_set_model = df
        else:
            data_set_model = data_set_model.join(df).sort_index(ascending=False)
    
    if write_files==True:
        path = config('PATH_MY_MAIN_DATA')
        data_set_full.to_csv(path_or_buf=path+'/_data_set_full')
        data_set_model.to_csv(path_or_buf=path+'/_data_set_model')

            
    return data_set_full, data_set_model
            
data_full, data_model = concat_save_data(df_list)